<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_multiple_choices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -U evaluate
import evaluate
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer,AutoModelForMultipleChoice,DataCollatorWithPadding,Trainer,TrainingArguments,pipeline
from transformers import BertForSequenceClassification, BertPreTrainedModel, BertModel
import torch
import numpy as np
from typing import Any

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
datasets=load_dataset("c3",'dialog')
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
model = AutoModelForMultipleChoice.from_pretrained("hfl/chinese-macbert-base")
datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at h

DatasetDict({
    train: Dataset({
        features: ['documents', 'document_id', 'questions'],
        num_rows: 4885
    })
    test: Dataset({
        features: ['documents', 'document_id', 'questions'],
        num_rows: 1627
    })
    validation: Dataset({
        features: ['documents', 'document_id', 'questions'],
        num_rows: 1628
    })
})

In [3]:
datasets["train"][:5]

{'documents': [['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'],
  ['男：足球比赛是明天上午八点开始吧?', '女：因为天气不好，比赛改到后天下午三点了。'],
  ['女：今天下午的讨论会开得怎么样?', '男：我觉得发言的人太少了。'],
  ['男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?', '女：是啊，我希望自己能瘦一点儿。'],
  ['女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。',
   '男：可不是嘛!自从我去日本留学，就再也没见过他，算一算都五年了。',
   '女：从2000年我们在学校第一次见面到现在已经快十年了。我还真想看看刘明变成什么样了!',
   '男：你还别说，刘明肯定跟英国绅士一样，也许还能带回来一个英国女朋友呢。']],
 'document_id': ['25-35', '31-109', '49-113', '12-53', '46-235'],
 'questions': [{'question': ['女的最喜欢哪种电影?'],
   'answer': ['喜剧片'],
   'choice': [['恐怖片', '爱情片', '喜剧片', '科幻片']]},
  {'question': ['根据对话，可以知道什么?'],
   'answer': ['比赛时间变了'],
   'choice': [['今天天气不好', '比赛时间变了', '校长忘了时间']]},
  {'question': ['关于这次讨论会，我们可以知道什么?'],
   'answer': ['讨论得不热烈'],
   'choice': [['会是昨天开的', '男的没有参加', '讨论得不热烈', '参加的人很少']]},
  {'question': ['女的为什么不吃巧克力了?'],
   'answer': ['要减肥'],
   'choice': [['刷牙了', '要减肥', '口渴了', '吃饱了']]},
  {'question': ['现在大概是哪一年?'],
   'answer': ['2010年'],
   'choice': [['2005年', '2010年',

In [4]:
datasets.pop("test")

Dataset({
    features: ['documents', 'document_id', 'questions'],
    num_rows: 1627
})

In [5]:
def data_process(examples):
  # examples是字典,keys=["documents", "quesitons"]
  #其中questions是含多个字典的列表，列表中每个字典的keys=["question","answer","choice"]
  context = []
  question_and_choice = []
  labels = []
  for i in range(len(examples["documents"])):
    ctx="\n".join(examples["documents"][i][0])
    question=examples["questions"][i]["question"][0]
    choices = examples["questions"][i]["choice"][0]
    for c in choices:
      context.append(ctx)
      question_and_choice.append(question+" "+c)
    if len(choices)<4:#choice是列表，最长长度是4，但不全是4，为了维度一致要对choice进行填充
      for _ in range(4-len(choices)):
        context.append(ctx)
        question_and_choice.append(question+" "+" ")
    labels.append(choices.index(examples["questions"][i]["answer"][0]))
  tokenized_examples=tokenizer(context,question_and_choice,truncation="only_first",max_length=128,padding="max_length")
  tokenized_examples={k:[v[i:i+4]for i in range(0,len(v),4)]for k ,v in tokenized_examples.items()}
  tokenized_examples["labels"] = labels
  return tokenized_examples
# 数据处理成：context,question+choice (都*4,因为最多有4个选项) 但label不*4
# 最终return出去的数据格式应该是：batch,num_choices,seq_length（这是模型要求的）
# {input_ids:[[Achoice1,Achoice2,Achoice3,Achoice4],[Bchoice1,Bchoice2,Bchoice3,Bchoice4],...],attention_msak:...,}


In [6]:
tmp=datasets["train"].select(range(10)).map(data_process, batched=True)
tmp

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['documents', 'document_id', 'questions', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [7]:
np.array(tmp["input_ids"]).shape

(10, 4, 128)

In [8]:
tokenized_ds = datasets.map(data_process, batched=True)

In [9]:
# def metric(pred)这里面的pred是np.array,pred[0]是predictions,pred[1]是labels
accuracy=evaluate.load("accuracy")

def metric(pred):
  predictions,labels=pred
  predictions=np.argmax(predictions,axis=-1)
  return accuracy.compute(predictions=predictions,references=labels)

In [10]:
# args = TrainingArguments(
#     output_dir="./muliple_choice",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=1,
#     logging_steps=50,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     fp16=True
# )

In [11]:
args = TrainingArguments(output_dir="./muliple_choice",      # 输出文件夹
                per_device_train_batch_size=4,  # 训练时的batch_size
                per_device_eval_batch_size=32,  # 验证时的batch_size
                logging_steps=10,                # log 打印的频率
                evaluation_strategy="steps",     # 评估策略
                eval_steps=20,
                save_steps=10,           # 保存策略
                num_train_epochs=1)

In [12]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=metric
)

In [ ]:
trainer.train()

In [14]:
class MultipleChoicePipeline:
  def __init__(self,model,tokenizer) -> None:
    self.model=model
    self.tokenizer=tokenizer
    self.device=self.model.device
  def preprocess(self,context,question,choices):
    ctx=[]
    que_and_ans=[]
    for choice in choices:
      ctx.append(context)
      que_and_ans.append(question+" "+choice)
    return tokenizer(ctx,que_and_ans,truncation="only_first",max_length=128,return_tensors="pt")
# tokenizer(ctx, que_and_ans, truncation="only_first", max_length=256, return_tensors="pt")
# tokenizer如果不指定return_tensor的话返回的就是列表，而且是在cpu上，而且单纯tokenizer出来的数据没有batch的维度，只有进行map(batched=True)才会有batch这个维度，所以用unsqueeze(0)给它加一个batch的维度
# 因为return的直接是pytorch的tensor，所以在post_process的时候使用的不是pred=np.argmax(predictions,axis=-1)而是pred=torch.argmax(logits,dim=-1).cpu.item()
# 为什么在预处理的时候不用return_tensors=呢？因为我们训练的时候传入了默认的datacollator,它自动转成tensor了
# 在自定义用于预测的pipeline的时候可以不需要padding（占资源），但在data_process的时候为了维度一定需要padding
  def predict(self,inputs):
    inputs={k:v.unsqueeze(0).to(self.device)for k,v in inputs.items()}
    #.unsqueeze(0)是为了给数据升维，有一个batch的维度
    return self.model(**inputs).logits
  def post_process(self,logits,choices):
    prediction=torch.argmax(logits,dim=-1).cpu().item()
    return choices[prediction]
  def __call__(self,context,question,choices) -> Any:
    inputs=self.preprocess(context,question,choices)
    logits=self.predict(inputs)
    result=self.post_process(logits,choices)
    return result

In [16]:
model = AutoModelForMultipleChoice.from_pretrained("/content/drive/MyDrive/nlp/muliple_choice/checkpoint-10")
pipe = MultipleChoicePipeline(model, tokenizer)
pipe("小明在北京上班", "小明在哪里上班？", ["北京", "上海", "河北", "海南", "河北", "海南"])
# 虽然训的时候最多只有4个选项，但是用的时候由于没有限制batch和num_choices，所以也可以给多于4个选项让模型进行选择

'上海'